
# **Modeling Flu Spread in Bucharest using Monte Carlo Simulation** #


##### The aim of this project is to model the spread of the flu in Bucharest using the Monte Carlo method to understand the dynamics of a virus in a crowded city. #####

In [57]:
import numpy as np
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

##### In this simulation model for Bucharest, we used a sample population of 10,000 individuals, with an estimated probability of infection between 3-5% per contact and an average recovery time of 7-10 days. The simulation is carried out over a 90-day period to cover a typical influenza season, allowing to observe the evolution of the spread of infection in a large city. ##### 

In [58]:
# Parameters for the simulation
nr_populatie = 10000
prob_infectare = np.random.uniform(0.03, 0.06, nr_populatie)  # Probability of infection per individual 
nr_zile_refacere = np.zeros(nr_populatie, dtype=int)         
simulare_zile = 150                                           # Number of simulation days (1 nov - 31 mar)
nr_sectoare = 6
nr_simulari = 1000 

In [59]:
# Simulation of the spread of the virus in the population every day
nr_infectati_pe_zi = []

for zi in range(simulare_zile):
    posibil_infectati = nr_zile_refacere == 0  # Not currently recovering
    sansa_infectare = np.random.rand(nr_populatie) 
    infectati = posibil_infectati & (sansa_infectare < prob_infectare)  # If random number is less than infection probability
    nr_infectati_pe_zi.append(np.sum(infectati))

    # Set recovery days for newly infected individuals
    nr_zile_refacere[infectati] = np.random.randint(7, 11, size=np.sum(infectati))

    # Decrease recovery days for all those recovering
    nr_zile_refacere[nr_zile_refacere > 0] -= 1

# Display results
print("Nr infectati pe zi: ", nr_infectati_pe_zi)



# Probability Estimation Using Monte Carlo Method
infected_count = 0  # Count of favorable outcomes

# Run trials for estimating infection probability
for _ in range(nr_simulari):
    # Simulate each individual's infection status
    for i in range(nr_populatie):
        random_value = np.random.rand()  # Generate a random number for this individual
        if random_value < prob_infectare[i]:  # Compare with individual infection probability
            infected_count += 1  # Increment count if infected

# Estimate probability
estimated_probability = infected_count / (nr_simulari * nr_populatie)  
print(f"Estimated Probability of Infection: {estimated_probability:.4f}")

Nr infectati pe zi:  [458, 429, 383, 402, 388, 356, 326, 311, 342, 322, 342, 319, 346, 351, 313, 318, 372, 327, 337, 332, 355, 325, 335, 346, 341, 330, 334, 318, 335, 330, 304, 356, 319, 311, 352, 348, 346, 317, 343, 321, 372, 308, 331, 327, 324, 359, 344, 342, 322, 345, 321, 368, 334, 361, 319, 337, 321, 319, 277, 350, 337, 350, 334, 344, 339, 313, 354, 365, 323, 331, 349, 328, 342, 341, 347, 329, 323, 338, 321, 342, 349, 337, 356, 331, 293, 342, 326, 313, 337, 371, 295, 357, 332, 328, 323, 330, 289, 372, 348, 356, 308, 321, 352, 320, 339, 353, 326, 337, 326, 344, 328, 349, 335, 352, 318, 334, 327, 326, 326, 378, 358, 360, 340, 332, 346, 337, 352, 318, 338, 331, 324, 337, 317, 304, 331, 343, 300, 321, 348, 352, 318, 327, 326, 326, 336, 366, 312, 308, 323, 294]
Estimated Probability of Infection: 0.0450


In [60]:
# Plotting the evolution of infected numbers each day
zile_pe_luna = [30, 31, 31, 28, 31]
luni = ['Nov', 'Dec', 'Jan', 'Feb', 'Mar'] 
pozitie_luna = [i * zile_pe_luna[i] for i in range(len(zile_pe_luna))]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(simulare_zile), 
    y=nr_infectati_pe_zi, 
    mode='lines', 
    name='Numar infectati in perioda noiembrie - martie', 
    line=dict(color='MediumPurple', width=2)))

fig.update_layout(title=dict(text='Evolutia gripei in Bucuresti (noiembrie - martie)', x=0.5),
                   xaxis=dict(title='Zile', tickvals=pozitie_luna, ticktext=luni),
                   yaxis_title='Nr persoane infectate',
                   paper_bgcolor='rgba(0,0,0,0)',
                   font=dict(family='system-ui', size=14, color='white'),
                   plot_bgcolor='Gainsboro', )
fig.show()

In [61]:
# Simulation parameters per sector
populatie_simulata_sectoare = np.array([1192, 1745, 2244, 1546, 1422, 1849])  # Simulated population for each sector
prob_infectare_sectoare = np.random.uniform(0.03, 0.05, nr_sectoare)          # Infection probability per sector

# Initialize recovery arrays for each sector
nr_zile_refacere_sectoare = [np.zeros(pop, dtype=int) for pop in populatie_simulata_sectoare]

# List to hold the number of infected individuals per day for each sector
nr_infectati_pe_zi_sectoare = [[] for _ in range(nr_sectoare)]

for zi in range(simulare_zile):
    for s in range(nr_sectoare):
        # Determine those who are susceptible to infection (not recovering)
        posibil_infectati = nr_zile_refacere_sectoare[s] == 0
        sansa_infectare = np.random.rand(populatie_simulata_sectoare[s])

        # Determine new infections
        infectati = posibil_infectati & (sansa_infectare < prob_infectare_sectoare[s])
        nr_infectati_pe_zi_sectoare[s].append(np.sum(infectati))

        # Set recovery days for newly infected individuals
        nr_zile_refacere_sectoare[s][infectati] = np.random.randint(7, 11, size=np.sum(infectati))

        # Decrement recovery days for those who are still in recovery
        nr_zile_refacere_sectoare[s][nr_zile_refacere_sectoare[s] > 0] -= 1

# Display results for each district
for s in range(nr_sectoare):
    print(f"Nr infectati pe zi in sectorul {s+1}: {nr_infectati_pe_zi_sectoare[s]}\n")


Nr infectati pe zi in sectorul 1: [51, 51, 37, 38, 48, 45, 36, 49, 37, 33, 27, 52, 35, 38, 42, 34, 30, 32, 32, 33, 30, 36, 47, 39, 28, 30, 35, 36, 37, 44, 36, 36, 38, 45, 35, 44, 32, 50, 37, 44, 43, 32, 34, 31, 44, 38, 49, 42, 37, 41, 38, 30, 32, 42, 41, 34, 44, 46, 36, 42, 32, 40, 48, 39, 38, 40, 27, 31, 41, 39, 31, 39, 31, 37, 33, 48, 37, 43, 47, 32, 43, 27, 31, 38, 40, 45, 29, 33, 33, 41, 41, 35, 30, 39, 45, 36, 28, 40, 46, 41, 33, 28, 29, 39, 39, 34, 38, 48, 33, 43, 39, 54, 32, 37, 36, 42, 45, 39, 47, 30, 45, 47, 37, 32, 43, 38, 36, 44, 35, 33, 51, 38, 40, 37, 45, 35, 30, 49, 39, 38, 45, 37, 37, 33, 40, 34, 40, 44, 42, 29]

Nr infectati pe zi in sectorul 2: [60, 43, 52, 57, 56, 47, 59, 52, 47, 36, 44, 47, 44, 50, 40, 46, 39, 51, 52, 54, 50, 56, 50, 38, 52, 48, 52, 44, 45, 55, 49, 46, 53, 44, 64, 48, 54, 53, 49, 44, 47, 47, 52, 39, 49, 45, 47, 47, 46, 37, 48, 40, 45, 57, 48, 42, 46, 45, 49, 41, 42, 45, 61, 44, 47, 41, 58, 40, 48, 43, 49, 47, 55, 49, 58, 49, 52, 60, 48, 41, 43, 43, 4

In [62]:
# Calculate total infections for each sector
infectii_pe_sector_toate_zilele = [sum(nr_infectati_pe_zi_sectoare[s]) for s in range(nr_sectoare)]

# Calculate infection rates for each sector
infectie_rate_sectoare = [infectii / populatie_simulata_sectoare[s] for s, infectii in enumerate(infectii_pe_sector_toate_zilele)] 

# Define sector labels that match the GeoJSON
idx_sector = ['Sector 1', 'Sector 2', 'Sector 3', 'Sector 4', 'Sector 5', 'Sector 6']

# Create a DataFrame with the sector labels and infection rates
data_rate = pd.DataFrame({'Sector': idx_sector, 'Infection Rate': infectie_rate_sectoare})
print(data_rate)

# Load the GeoJSON file for Bucharest sectors
geojson_fisier = '/Users/alexandrazamfirescu/Bucharest_Streets/Flu_Spread_Simulation/BucurestiSectoare.geo.json'

with open(geojson_fisier) as f:
    geojson = json.load(f)

# Plot the choropleth map for infection rates
fig = px.choropleth(
    data_frame=data_rate,
    geojson=geojson_fisier,
    locations='Sector',
    featureidkey='properties.tags.name',  # Ensure this matches your GeoJSON
    color='Infection Rate',
    color_continuous_scale='Turbo',
    labels={'Infection Rate': 'Infection Rate'},
    projection='mercator'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title=dict(text='Rata de infectare cu gripa in fiecare sector', x=0.5),
    margin={"r":0, "t":40, "l":0, "b":0}
)
fig.show()


     Sector  Infection Rate
0  Sector 1        4.821309
1  Sector 2        4.135244
2  Sector 3        5.153743
3  Sector 4        4.286546
4  Sector 5        3.759494
5  Sector 6        3.994051
